In [6]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from xgboost import XGBRegressor
from sklearn.ensemble import StackingClassifier, GradientBoostingClassifier


In [3]:
df = pd.read_csv('../Datasets/cases/Sonar/Sonar.csv')
X = df.drop('Class', axis=1)
y = df['Class']
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25, stratify=y)

rates = np.linspace(0.01, 0.8, 20)
n_est = np.arange(50, 200, 30)
depths = [None, 2, 3, 4, 5, 7]
scores = []

In [7]:
dtc = DecisionTreeClassifier(random_state=23)
knn = KNeighborsClassifier()
gnb = GaussianNB()
gbm = GradientBoostingClassifier(random_state=23)

stack = StackingClassifier([
    ('Tree', dtc),
    ('knn', knn),
    ('GaussianNB', gnb)
], final_estimator=gbm)

stack.fit(X_train, y_train)
y_pred_prob = stack.predict_proba(X_test)
y_pred = stack.predict(X_test)


In [10]:
print('log_loss ', log_loss(y_test, y_pred_prob))
print('accuracy ', accuracy_score(y_test, y_pred))

log_loss  0.7317819085619308
accuracy  0.6984126984126984


with passthrough

In [11]:
dtc = DecisionTreeClassifier(random_state=23)
knn = KNeighborsClassifier()
gnb = GaussianNB()
gbm = GradientBoostingClassifier(random_state=23)


stack = StackingClassifier([
    ('dtc', dtc),
    ('knn', knn),
    ('gnb', gnb)
], final_estimator=gbm,
passthrough=True)

stack.fit(X_train, y_train)
y_pred_prob = stack.predict_proba(X_test)
y_pred = stack.predict(X_test)


In [12]:
print('log_loss ', log_loss(y_test, y_pred_prob))
print('accuracy ', accuracy_score(y_test, y_pred))

log_loss  0.48318321958961497
accuracy  0.7777777777777778
